# Database data validation

In this notebook, we will see how we can use the great_expectations package to validate data in our database.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NatanMish/data_validation/blob/main/notebooks/1_database_data_validation.ipynb)

#### Install the required packages and import them to the notebook


In [3]:
!pip install -U great_expectations pandas

  Using cached great_expectations-0.15.10-py3-none-any.whl (5.1 MB)
  Using cached pandas-1.4.2-cp38-cp38-macosx_10_9_x86_64.whl (11.0 MB)
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached nbformat-5.4.0-py3-none-any.whl (73 kB)
  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)
  Using cached tzlocal-4.2-py3-none-any.whl (19 kB)
  Using cached mistune-2.0.2-py2.py3-none-any.whl (24 kB)
  Using cached colorama-0.4.5-py2.py3-none-any.whl (16 kB)
  Using cached jsonschema-4.6.0-py3-none-any.whl (80 kB)
  Using cached pytz-2022.1-py2.py3-none-any.whl (503 kB)
  Using cached importlib_metadata-4.11.4-py3-none-any.whl (18 kB)
  Using cached altair-4.2.0-py3-none-any.whl (812 kB)
  Using cached typing_extensions-4.2.0-py3-none-any.whl (24 kB)
  Using cached numpy-1.22.4-cp38-cp38-macosx_10_15_x86_64.whl (17.6 MB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
     |████████████████████████████████| 55.3 MB 836 kB/s eta 0:00:01
  Using cached jsonpatch-1.32-

In [4]:
# import the required packages
import great_expectations as ge
from ruamel import yaml
import pandas as pd

[![Great Expectations](https://docs.greatexpectations.io/img/great-expectations-long-logo.svg)]

Named after the famous 19th century novel, Great Expectations is a shared, open sourced package for data quality. It helps eliminate pipeline debt, through data testing, documentation, and profiling. It is a tool for data scientists, data engineers, and data analysts to validate data. GE has many useful integrations and can be connected directly to SQL databases, Apache Spark, Apache Airflow, Bigquery, and more. In this tutorial, we will validate a database hosted on a  local file system, but the process for a cloud file system such as a Data Lake, Azure Blob Storage, GCP bucket or AWS S3 is almost identical.

**Terminology**
1. *Data Context* - The primary entry point for a Great Expectations deployment, with configurations and methods for all supporting components.

2. *Data Source* - Provides a standard API for accessing and interacting with data from a wide variety of source systems.

3. *Data Asset* - A collection of records within a Datasource which is usually named based on the underlying data system and sliced to correspond to a desired specification.

4. *Expectation Suite* - A collection of verifiable assertions about data.

5. *Validation* - The act of applying an Expectation Suite to a Batch.

6. *Batch Identifier* - contains information that uniquely identifies a specific batch from the Data Asset, such as the delivery date or query time.

7. *Data Connector* - Provides the configuration details based on the source data system which are needed by a Datasource to define Data Assets.

### 1. Create a Data Context

We will now create a data context, which is the first step in setting up Great Expectations for our project. Creating a data context is actually most easily done in bash using the great_expectations CLI. Run the shell command below and this will initialize a new data context in the current directory. The `echo y` bit is used to suppress the interactive prompt. You will now see a new directory called `great_expectations` created in your current directory.

In [5]:
!echo y | great_expectations init

Using v3 (Batch Request) API

  ___              _     ___                  _        _   _
 / __|_ _ ___ __ _| |_  | __|_ ___ __  ___ __| |_ __ _| |_(_)___ _ _  ___
| (_ | '_/ -_) _` |  _| | _|\ \ / '_ \/ -_) _|  _/ _` |  _| / _ \ ' \(_-<
 \___|_| \___\__,_|\__| |___/_\_\ .__/\___\__|\__\__,_|\__|_\___/_||_/__/
                                |_|
             ~ Always know what to expect from your data ~

Let's create a new Data Context to hold your project configuration.

Great Expectations will create a new directory with the following structure:

    great_expectations
    |-- great_expectations.yml
    |-- expectations
    |-- checkpoints
    |-- plugins
    |-- .gitignore
    |-- uncommitted
        |-- config_variables.yml
        |-- data_docs
        |-- validations

OK to proceed? [Y/n]: 

Congratulations! You are now ready to customize your Great Expectations configuration.

You can customize your configuration in many ways. Here are some examples:

  Use the CLI to:
    - Ru

After running the init command, your great_expectations directory will contain all the important components of a local Great Expectations deployment. This is what the directory structure looks like:

- `great_expectations.yml` contains the main configuration of your deployment.
The expectations directory stores all your Expectations as JSON files. If you want to store them somewhere else, you can change that later.

- The `plugins/` directory holds code for any custom plugins you develop as part of your deployment.

- The `uncommitted/` directory contains files that shouldn’t live in version control. It has a .gitignore configured to exclude all its contents from version control. The main contents of the directory are:
    1. `uncommitted/config_variables.yml`, which holds sensitive information, such as database credentials and other secrets.
    2. `uncommitted/data_docs`, which contains Data Docs generated from Expectations, Validation Results, and other metadata.
    3. `uncommitted/validations`, which holds Validation Results generated by Great Expectations.

<div>
<img src="https://docs.greatexpectations.io/assets/images/data_context_does_for_you-df2eca32d0152ead16cccd5d3d226abb.png" width="1000"/>
</div>

### 2. Create a Data Source

In [6]:
# We will start by reading in the GE data context we have created in the previous step
context = ge.get_context()

Now we will script a yaml file to create a data source. We will need the following configuration parameters:

In [7]:
datasource_name = "house_prices"
# Data Source - Provides a standard API for accessing and interacting with data from a wide variety of source systems.

In [8]:
execution_engine = "PandasExecutionEngine"  # alternatively we can use SparkExecutionEngine for PySpark oriented
# projects or SqlAlchemyExecutionEngine for creating a SQL database data source.
data_directory = "data"

In [9]:
data_asset_name = f"{datasource_name}_survey_2006"
# Data Asset - A collection of records within a Datasource which is usually named based on the underlying data system and sliced to correspond to a desired specification.

In [10]:
runtime_data_connector_name = "runtime_batch_files_connector"
# Data Connector - Provides the configuration details based on the source data system which are needed by a Datasource to define Data Assets.

In [11]:
datasource_config = {
    "name": datasource_name,
    "class_name": "Datasource",
    "module_name": "great_expectations.datasource",
    "execution_engine": {
        "module_name": "great_expectations.execution_engine",
        "class_name": execution_engine,
    },
    "data_connectors": {
        runtime_data_connector_name: {
            "class_name": "RuntimeDataConnector",
            "batch_identifiers": ["default_identifier_name"],
        },
    },
}

In [12]:
# Test that the configuration is valid
context.test_yaml_config(yaml.dump(datasource_config))

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: PandasExecutionEngine
Data Connectors:
	runtime_batch_files_connector:RuntimeDataConnector

	Available data_asset_names (0 of 0):
		Note : RuntimeDataConnector will not have data_asset_names until they are passed in through RuntimeBatchRequest

	Unmatched data_references (0 of 0): []



/Users/samia/opt/anaconda3/envs/pydatalondon2022/lib/python3.8/site-packages/great_expectations/datasource/data_connector/runtime_data_connector.py:133: DeprecationWarning: Specifying batch_identifiers as part of the RuntimeDataConnector config is deprecated as of v0.15.1 and will be removed by v0.18. Please configure batch_identifiers as part of Assets instead.
  warnings.warn(


In [13]:
# If the configuration is valid, we can create the datasource
context.add_datasource(**datasource_config)

In [14]:
# Now we can see that the datasource was created.
context.list_datasources()

[{'data_connectors': {'runtime_batch_files_connector': {'module_name': 'great_expectations.datasource.data_connector',
    'class_name': 'RuntimeDataConnector',
    'batch_identifiers': ['default_identifier_name']}},
  'module_name': 'great_expectations.datasource',
  'class_name': 'Datasource',
  'execution_engine': {'module_name': 'great_expectations.execution_engine',
   'class_name': 'PandasExecutionEngine'},
  'name': 'house_prices'}]

### 3. Create an Expectation Suite
Expectations are the core of Great Expectations. They are the assertions that are used to validate data. Let's create an expectation suite which is a collection of expectations. This diagram below shows how we can define good expectations for our data.

<div>
<img src="https://docs.greatexpectations.io/assets/images/where_expectations_come_from-b3504cf51ad304c8e4a73677a0e73156.png" width="1000"/>
</div>

We will create expectations while exploring the data in the notebook. The method below behaves  exactly the same as `pandas.read_csv`. Similarly wrapped versions of other pandas methods (`read_excel`, `read_table`, `read_parquet`, `read_pickle`, `read_json`, etc.) are also available.

In [15]:
home_data = ge.read_csv("https://github.com/NatanMish/data_validation/blob/a77b247b25c6622ce0c8f8cbc505228161c31a3c/data/train.csv?raw=true")

In [16]:
# The home_data variable is a pandas dataframe with all the methods and properties we know and love. We can use the `head` method to see the
# first few rows of the data.
home_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
# beyond the Pandas methods and properties, we can use GE's expectations methods to define expectations. 
# In Jupyter, type in `home_data.expect` and press tab to see the list of available expectations.
# home_data.expect

home_data.expect

In [17]:
# Let's create a few example expectations and see if they are valid on this dataset.
home_data.expect_column_to_exist("Id")

{
  "success": true,
  "result": {},
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Notice the `"success": true` key in the result dictionary, this means the expectation is valid for this data source

In [18]:
home_data.expect_column_values_to_be_unique("Id")

{
  "success": true,
  "result": {
    "element_count": 1460,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

The expectation above checked the contents of the column, hence we got a few other useful metrics, showing how many 
rows were inspected, how many were missing etc.

In [19]:
# This expectation should fail, lets see what happens:
home_data.expect_column_max_to_be_between("SalePrice", 0, 100000)

{
  "success": false,
  "result": {
    "observed_value": 755000,
    "element_count": 1460,
    "missing_count": null,
    "missing_percent": null
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

The returned dictionary shows that the expectation is not valid, and the value observed that is not in the expected range.
Here are a few more useful expectation definitions:

In [20]:
home_data.expect_column_distinct_values_to_be_in_set("MSZoning", ["C (all)", "FV", "RH", "RL", "RM"])

{
  "success": true,
  "result": {
    "observed_value": [
      "C (all)",
      "FV",
      "RH",
      "RL",
      "RM"
    ],
    "element_count": 1460,
    "missing_count": null,
    "missing_percent": null
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [21]:
home_data.expect_column_mean_to_be_between("GrLivArea", 0, 10000)

{
  "success": true,
  "result": {
    "observed_value": 1515.463698630137,
    "element_count": 1460,
    "missing_count": null,
    "missing_percent": null
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [22]:
# This will create an expectation suite from all the valid expectations we created above.
home_data.get_expectation_suite()
# If we want the non-valid expectations as well, we can use the `get_expectation_suite` method with the 
# `discard_failed_expectations` parameter set to True. If there are any duplicate expectations in the suite, 
# the duplicates will be discarded:
# home_data.get_expectation_suite(discard_failed_expectations=False)

{
  "expectation_suite_name": "default",
  "ge_cloud_id": null,
  "expectations": [
    {
      "kwargs": {
        "column": "Id"
      },
      "meta": {},
      "expectation_type": "expect_column_to_exist"
    },
    {
      "kwargs": {
        "column": "Id"
      },
      "meta": {},
      "expectation_type": "expect_column_values_to_be_unique"
    },
    {
      "kwargs": {
        "column": "MSZoning",
        "value_set": [
          "C (all)",
          "FV",
          "RH",
          "RL",
          "RM"
        ]
      },
      "meta": {},
      "expectation_type": "expect_column_distinct_values_to_be_in_set"
    },
    {
      "kwargs": {
        "column": "GrLivArea",
        "min_value": 0,
        "max_value": 10000
      },
      "meta": {},
      "expectation_type": "expect_column_mean_to_be_between"
    }
  ],
  "meta": {
    "great_expectations_version": "0.15.10"
  },
  "data_asset_type": "Dataset"
}

In [23]:
# This line will save the expectation suite to the data context
context.save_expectation_suite(home_data.get_expectation_suite(), "my_expectations")

'/Users/samia/Downloads/PyDataLondon2022/Friday_TS2_0900/notebooks/great_expectations/expectations/my_expectations.json'

In [26]:
home_data.expect

AttributeError: 'PandasDataset' object has no attribute 'expect'

### Exercise 1
Check the following expectations to see if they are valid on the home_data dataframe:

(Not all the expectations were included in the examples above. You can find more expectations in the [expectations directory](https://greatexpectations.io/expectations).)
1. `Street` column should be a string.
2. `LandContour` column cannot be null.
3. `YearBuilt` minimal value should be between 1700 and 1900.
4. `LotArea` median value should be between 5000 and 15000.
5. The most common values in `SaleType` must be either `WD` or `New`.

In [37]:
# your answers here:
#home_data.expect_column_values_to_be_of_type('Street', 'str')
#home_data.expect_column_values_to_not_be_null('LandContour')

# home_data.expect_column_values_to_be_between('YearBuilt', 1700, 1900)


# home_data.expect_column_median_to_be_between('LotArea', 5000, 15000)
# home_data.expect_column_median_to_be_between('LotArea', 5000, 15000)
home_data.expect_column_most_common_value_to_be_in_set('SaleType', ["WD", "New"])

#most common value to be in set, 'salestype, ["t", "3"]
# home_data.expec
# home_data.ex
#home_data.exp
# home_data.expect_column_
# home_data.expect_column_
# home_data.expect_column_
# home_data.expect_column_

{
  "success": true,
  "result": {
    "observed_value": [
      "WD"
    ],
    "element_count": 1460,
    "missing_count": null,
    "missing_percent": null
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

*Exercise solutions can be found in the exercise solutions file in the current directory.*

### 4. Validate the Data
We will now validate the test data using the expectations we have created for the train data.

In [38]:
checkpoint_name = "data_batch_appended"
# Checkpoint - The primary means for validating data in a production deployment of Great Expectations.

In [39]:
checkpoint_config = {
    "name": checkpoint_name,
    "config_version": 1,
    "class_name": "SimpleCheckpoint",
    "validations": [
        {
            "batch_request": {
                "datasource_name": datasource_name,
                "data_connector_name": runtime_data_connector_name,
                "data_asset_name": data_asset_name,
            },
            "expectation_suite_name": "my_expectations",
        }
    ],
}
context.add_checkpoint(**checkpoint_config)

{
  "action_list": [
    {
      "name": "store_validation_result",
      "action": {
        "class_name": "StoreValidationResultAction"
      }
    },
    {
      "name": "store_evaluation_params",
      "action": {
        "class_name": "StoreEvaluationParametersAction"
      }
    },
    {
      "name": "update_data_docs",
      "action": {
        "class_name": "UpdateDataDocsAction",
        "site_names": []
      }
    }
  ],
  "batch_request": {},
  "class_name": "Checkpoint",
  "config_version": 1.0,
  "evaluation_parameters": {},
  "module_name": "great_expectations.checkpoint",
  "name": "data_batch_appended",
  "profilers": [],
  "runtime_configuration": {},
  "validations": [
    {
      "batch_request": {
        "datasource_name": "house_prices",
        "data_connector_name": "runtime_batch_files_connector",
        "data_asset_name": "house_prices_survey_2006"
      },
      "expectation_suite_name": "my_expectations"
    }
  ]
}

Looking at the dictionary returned by the `add_checkpoint` methods we can see what are the actions performed every time the checkpoint will run:
1. Store validation result.
2. Store evaluation parameters.
3. Update data docs. (we will look at the data docs later in this notebook)

In [40]:
home_data_test = pd.read_csv("https://github.com/NatanMish/data_validation/blob/a77b247b25c6622ce0c8f8cbc505228161c31a3c/data/test.csv?raw=true")

In [41]:
results = context.run_checkpoint(
    checkpoint_name=checkpoint_name,
    batch_request={
        "runtime_parameters": {"batch_data": home_data_test},
        "batch_identifiers": {
            "default_identifier_name": "default_identifier_name"
        },
    },
)
# Batch Identifier - contains information that uniquely identifies a specific batch from the Data Asset, such as the delivery date or query time.

Calculating Metrics: 100%|██████████| 10/10 [00:00<00:00, 465.88it/s]


In [42]:
# Let's take a look at the validation result object we got:
run_identifier = next(iter(results['run_results']))
results['run_results'][run_identifier]['validation_result']['statistics']

{'evaluated_expectations': 4,
 'successful_expectations': 4,
 'unsuccessful_expectations': 0,
 'success_percent': 100.0}

In [43]:
# Here is an example of one of the validations on one of the expectations. The check has passed and there are some 
# useful extra details too.
results['run_results'][run_identifier]['validation_result']['results'][2]

{
  "success": true,
  "result": {
    "observed_value": [
      "C (all)",
      "FV",
      "RH",
      "RL",
      "RM"
    ],
    "details": {
      "value_counts": [
        {
          "value": "C (all)",
          "count": 15
        },
        {
          "value": "FV",
          "count": 74
        },
        {
          "value": "RH",
          "count": 10
        },
        {
          "value": "RL",
          "count": 1114
        },
        {
          "value": "RM",
          "count": 242
        }
      ]
    }
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

#### How does an invalid data checkpoint look like?
Glad you asked, let's inject a duplicate value to our `Id` column to see how it behaves:

In [44]:
# This will create a duplicate id value for two separate records
home_data_test.at[0, 'Id'] = 1462

In [45]:
bad_data_checkpoint_name = "my_bad_data_checkpoint"
bad_data_checkpoint_config = {
    "name": bad_data_checkpoint_name,
    "config_version": 1,
    "class_name": "SimpleCheckpoint",
    "validations": [
        {
            "batch_request": {
                "datasource_name": datasource_name,
                "data_connector_name": runtime_data_connector_name,
                "data_asset_name": "batch_data_asset",
            },
            "expectation_suite_name": "my_expectations",
        }
    ],
}
context.add_checkpoint(**bad_data_checkpoint_config)

{
  "action_list": [
    {
      "name": "store_validation_result",
      "action": {
        "class_name": "StoreValidationResultAction"
      }
    },
    {
      "name": "store_evaluation_params",
      "action": {
        "class_name": "StoreEvaluationParametersAction"
      }
    },
    {
      "name": "update_data_docs",
      "action": {
        "class_name": "UpdateDataDocsAction",
        "site_names": []
      }
    }
  ],
  "batch_request": {},
  "class_name": "Checkpoint",
  "config_version": 1.0,
  "evaluation_parameters": {},
  "module_name": "great_expectations.checkpoint",
  "name": "my_bad_data_checkpoint",
  "profilers": [],
  "runtime_configuration": {},
  "validations": [
    {
      "batch_request": {
        "datasource_name": "house_prices",
        "data_connector_name": "runtime_batch_files_connector",
        "data_asset_name": "batch_data_asset"
      },
      "expectation_suite_name": "my_expectations"
    }
  ]
}

In [46]:
results_bad_data_checkpoint = context.run_checkpoint(
    checkpoint_name=bad_data_checkpoint_name,
    batch_request={
        "runtime_parameters": {"batch_data": home_data_test},
        "batch_identifiers": {
            "default_identifier_name": "default_identifier_name"
        },
    },
)

Calculating Metrics: 100%|██████████| 10/10 [00:00<00:00, 685.93it/s]


In [47]:
# As expected, not all expectations were successful.
bad_data_run_identifier = next(iter(results_bad_data_checkpoint['run_results']))
results_bad_data_checkpoint['run_results'][bad_data_run_identifier]['validation_result']['statistics']

{'evaluated_expectations': 4,
 'successful_expectations': 3,
 'unsuccessful_expectations': 1,
 'success_percent': 75.0}

In [48]:
# And here is the summary for the failed expectation
results_bad_data_checkpoint['run_results'][bad_data_run_identifier]['validation_result']['results'][1]

{
  "success": false,
  "result": {
    "element_count": 1459,
    "unexpected_count": 2,
    "unexpected_percent": 0.1370801919122687,
    "partial_unexpected_list": [
      1462,
      1462
    ],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.1370801919122687,
    "unexpected_percent_nonmissing": 0.1370801919122687,
    "partial_unexpected_index_list": null,
    "partial_unexpected_counts": [
      {
        "value": 1462,
        "count": 2
      }
    ]
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}